In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low= -90.000, high = 90.000, size = 2000)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
coordinates

[(-39.0546126498808, 2.811573910449141),
 (-25.915279297265386, 61.210779189536225),
 (-53.34015358881555, -92.78678824031668),
 (3.865197163265094, 178.2222319043608),
 (-12.338499248999682, -47.600856540918386),
 (-68.48741245325068, -119.44074663214408),
 (1.446769949411788, 109.70564778749485),
 (-55.95084317966554, 35.89482204405252),
 (79.38514747461466, 111.64806807588906),
 (9.440987942582979, 71.6723121717753),
 (-63.60463684237969, 85.89496975602384),
 (9.90551882658626, 91.51250678284833),
 (-69.97186579516244, 173.15202618285264),
 (69.05764459415448, -46.47305850247889),
 (-9.063027119160964, 15.586625964095589),
 (-72.7490502267153, -98.66780555528942),
 (-23.33522620603192, 114.5874413994652),
 (62.82630788840132, -92.00838547527223),
 (-45.69761817590609, 7.500039301611707),
 (-3.2819025611391197, -97.0399851381369),
 (66.5083609249875, -41.044286184277865),
 (-17.469960179241653, 63.52840132699072),
 (19.04433690689912, 38.88840836248616),
 (60.68335322888095, 49.08271

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

731

In [6]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [7]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
#cities = cities[0:10]
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
         # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather['weather'][0]['description']
        city_country = city_weather["sys"]["country"]
            # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                              "Lat": city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Humidity": city_humidity,
                              "Cloudiness": city_clouds,
                              'Current Description': city_weather_description,
                              "Wind Speed": city_wind,
                              "Country": city_country,
                              "Date": city_date})


    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | saldanha
Processing Record 2 of Set 1 | mahebourg
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | rawannawi
City not found. Skipping...
Processing Record 5 of Set 1 | parana
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | kuching
Processing Record 8 of Set 1 | port alfred
Processing Record 9 of Set 1 | saskylakh
Processing Record 10 of Set 1 | kavaratti
Processing Record 11 of Set 1 | busselton
Processing Record 12 of Set 1 | port blair
Processing Record 13 of Set 1 | bluff
Processing Record 14 of Set 1 | ilulissat
Processing Record 15 of Set 1 | malanje
Processing Record 16 of Set 1 | carnarvon
Processing Record 17 of Set 1 | thompson
Processing Record 18 of Set 1 | cape town
Processing Record 19 of Set 1 | puerto ayora
Processing Record 20 of Set 1 | tasiilaq
Processing Record 21 of Set 1 | quatre cocos
Processing Record 22 of Set 1 | tawkar
City

In [ ]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

In [ ]:
# Reorder the columns as City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, Windspeed
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity","Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

In [ ]:
# Create the output file (CSV).
output_data_file = "Weather_database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")